# Weakspot Analysis Example

This notebook demonstrates how to use the WeakspotComponent to identify and visualize data slices where your machine learning model performs poorly.

## What is Weakspot Analysis?

Weakspot analysis helps you identify regions in your feature space where your model performs significantly worse than its overall performance. This is useful for:

- Understanding model limitations
- Identifying potential bias
- Focusing improvement efforts
- Model validation and debugging

In [1]:
# Import required libraries
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

from explainerdashboard import ClassifierExplainer, ExplainerDashboard
from explainerdashboard.dashboard_components.weakspot_components import WeakspotComponent
from explainerdashboard.dashboard_components.plot_analysis_components import PlotAnalysisComponent
from explainerdashboard.datasets import titanic_survive,titanic_names,feature_descriptions

In [2]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

## Load and Prepare Data

We'll use the Titanic dataset to demonstrate weakspot analysis.

In [3]:
X_train, y_train, X_test, y_test = titanic_survive()
train_names, test_names = titanic_names()

## Train a Model

Let's train a Random Forest classifier on the Titanic data.

In [4]:
# Train a Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

## Create Explainer

Create a ClassifierExplainer to enable weakspot analysis.

In [5]:
# Create explainer
explainer = ClassifierExplainer(
    model, 
    X_test, 
    y_test,
    cats=["Sex", "Deck", "Embarked"],
    idxs=test_names,
    descriptions=feature_descriptions,
    target="Survival",
    labels=["Not survived", "Survived"],
)

print(f"Explainer created with {len(explainer.columns)} features")
print(f"Overall model accuracy: {explainer.metrics()['accuracy']:.3f}")

Detected RandomForestClassifier model: Changing class type to RandomForestClassifierExplainer...
Note: model_output=='probability', so assuming that raw shap output of RandomForestClassifier is in probability space...
Generating self.shap_explainer = shap.TreeExplainer(model)
Explainer created with 21 features
Calculating prediction probabilities...
Calculating metrics...
Overall model accuracy: 0.785


## Basic Weakspot Analysis

Let's perform a basic weakspot analysis using the explainer methods directly.

In [6]:
# Perform weakspot analysis on Age feature
result = explainer.calculate_weakspot_analysis(
    slice_features=['PassengerClass'],
    slice_method='histogram',
    bins=10,
    metric='accuracy',
    threshold=1.01,  # 20% worse than overall performance
    min_samples=10
)

print(f"Overall accuracy: {result['overall_metric']:.3f}")
print(f"Weakness threshold: {result['threshold_value']:.3f}")
print(f"Number of weak regions found: {len(result['weak_regions'])}")

if result['weak_regions']:
    print("\nWeak regions:")
    for i, region in enumerate(result['weak_regions']):
        # 1. Get the feature name (it's the first and only key in the dict)
        feature_name = list(region['range'].keys())[0]
        
        # 2. Get the range tuple using the feature name as the key
        range_tuple = region['range'][feature_name]

        # 3. Use these variables in your f-string
        print(f"  {i+1}. {feature_name} [{range_tuple[0]:.3f}-{range_tuple[1]:.3f}]: "
              f"accuracy = {region['performance']:.3f} "
              f"({region['sample_count']} samples)")
else:
    print("\nNo weak regions detected with current threshold.")

Calculating predictions...
Overall accuracy: 0.785
Weakness threshold: 1.010
Number of weak regions found: 1

Weak regions:
  1. PassengerClass [1.000-1.200]: accuracy = 0.731 (52 samples)


## Generate Summary

Get a text summary of the weakspot analysis.

In [7]:
# Generate summary
summary = explainer.weakspot_summary(
    slice_features=['PassengerClass'],
    slice_method='histogram',
    bins=10,
    metric='accuracy',
    threshold=1.01,
    min_samples=10
)

print("Weakspot Analysis Summary:")
print(summary)

Weakspot Analysis Summary:
Weakspot Analysis Summary
Features analyzed: PassengerClass
Slicing method: histogram
Performance metric: accuracy
Overall performance: 0.7850
Weakness threshold: 1.01x

Results:
- Total slices: 3
- Weak slices: 1
- Weak percentage: 33.3%
- Performance range: 0.1516

Weak Regions Found:
1. PassengerClass: [1.000, 1.200] - Performance: 0.7308 (Severity: 1.07x, Samples: 52)


## Interactive Dashboard with WeakspotComponent

Now let's create an interactive dashboard that includes the WeakspotComponent.

In [8]:
# Create a standalone WeakspotComponent
weakspot_component = WeakspotComponent(
    explainer,
    title="Titanic Model Weakspot Analysis",
    slice_features=['Age'],  # Start with Age analysis
    threshold=1.2,  # 20% worse performance threshold
    bins=10,
    min_samples=10
)

# Create dashboard with the weakspot component
dashboard = ExplainerDashboard(
    explainer,
    tabs=['whatif','weakspot'],
    title="Titanic Weakspot Analysis",
    description="Identify weak performance regions in the Titanic survival model"
)

print("Dashboard created! Run dashboard.run() to start the interactive dashboard.")

Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
Generating layout...
Calculating shap values...
Calculating dependencies...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...
Dashboard created! Run dashboard.run() to start the interactive dashboard.


In [9]:
dashboard.run(port=9000)

Starting ExplainerDashboard on http://192.168.2.91:9000


In [10]:
dashboard.terminate(port=9000)

Trying to shut down dashboard on port 9000...


## Key Features of WeakspotComponent

The WeakspotComponent provides several interactive features:

1. **Feature Selection**: Choose 1 or 2 features to analyze
2. **Slicing Methods**: 
   - Histogram: Equal-width bins
   - Decision Tree: Automatic optimal splits
3. **Performance Metrics**:
   - Classification: Accuracy, Log Loss, Brier Score
   - Regression: MSE, MAE, MAPE
4. **Configurable Parameters**:
   - Weakness threshold (how much worse than overall performance)
   - Number of bins (for histogram method)
   - Minimum samples per slice
5. **Interactive Visualizations**:
   - 1D plots for single feature analysis
   - 2D heatmaps for two-feature analysis
   - Hover information and export capabilities

## Tips for Effective Weakspot Analysis

1. **Start with important features**: Use SHAP or feature importance to identify key features
2. **Adjust the threshold**: Lower thresholds (1.1-1.2) find subtle weaknesses, higher thresholds (1.5+) find severe issues
3. **Consider sample sizes**: Ensure slices have enough samples for reliable metrics
4. **Try different slicing methods**: Histogram for uniform exploration, tree for optimal splits
5. **Analyze multiple features**: Two-feature analysis can reveal interaction effects

## Troubleshooting Common Issues

### No Weak Regions Found

If your analysis shows no weak regions, try:

- **Lower the threshold**: Use 1.05-1.1 instead of 1.2+ for more sensitive detection
- **Reduce minimum samples**: Try `min_samples=5` or `10` for smaller datasets
- **Increase bins**: Use 15-20 bins for more granular analysis
- **Try different features**: Some features may not have clear weak regions
- **Use tree slicing**: Decision tree method can find optimal splits

### Too Many Weak Regions

If nearly every slice is weak, try:

- **Increase the threshold**: Use 1.5+ to focus on severely underperforming regions
- **Increase minimum samples**: Use 30-50 for more reliable performance estimates
- **Reduce bins**: Use 5-8 bins for larger, more stable slices
- **Check data quality**: Widespread weak regions might indicate model or data issues

### Performance Issues

For large datasets:

- **Use sampling**: Analyze a representative sample instead of full data
- **Reduce bins**: Fewer bins mean less computation
- **Single feature analysis**: Two-feature analysis is more expensive
- **Use histogram method**: Tree-based slicing can be slower

## Next Steps

Once you've identified weak regions:

1. **Investigate the data**: Look at samples in weak regions to understand patterns
2. **Feature engineering**: Create new features to better capture these patterns
3. **Data collection**: Gather more training data for weak regions
4. **Model adjustments**: Consider ensemble methods or specialized models for weak regions
5. **Monitoring**: Set up monitoring to detect when new data falls into known weak regions